## My train using Learner

Training BART with a changed script from th GPT2 project

### !! PROBLEM: standard BART doesn't allow position_ids. Should we rewrite the code??
Hell yeah we should...

In [1]:
# #!pip uninstall transformers
# !python -m venv .env
# !source .env/bin/activate
#!pip install transformers==4.5.1
# !pip show transformers

#!pip install torch==1.8.1
# !pip install fairscale
#!pip install --user --upgrade papermill

In [2]:
# !pip show transformers
#!python -m pip list --user
# !pip show deepspeed


In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import argparse
from datetime import datetime
import json
import os
import pickle
import random
import time
import wandb
from pathlib import Path
import gc
import glob

import numpy as np
from transformers import BartForConditionalGeneration, GPT2Tokenizer, PreTrainedTokenizerBase, DataCollatorForLanguageModeling
from transformers import BartConfig, AdamW, get_linear_schedule_with_warmup, TrainingArguments, Trainer
from transformers.file_utils import PaddingStrategy, logging
from tensorboardX import SummaryWriter
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.cuda.amp import autocast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tnrange, tqdm

from dataset import SpectroDataset, SpectroDataCollator
sys.path.append('data')
from data_preprocess1 import print_args
# from utils import add_special_tokens, generate_sample, sample_seq, set_seed, top_k_top_p_filtering, print_args

#### Setting basic training args

In [4]:
!pwd

/auto/brno6/home/ahajek/Spektro/MassGenie


In [5]:
bs = 4 #4
gas = 16 #16
data_type = "1M"
SEQ_LEN = 200
num_epochs = 8 # int(os.environ["TOTAL_EPOCHS"]) #21 + 8 = BP
resume_training = False # bool(int(os.environ["RESUME_TRAINING"]))
# resume_wandb_id = pass # ""

# find the last checkpoint

# models_pth = "/storage/projects/msml/mg_neims_branch/models/bart_trial"
# runs = glob.glob(models_pth)
# checkpoints =  glob.glob(runs[-1]+"/checkpoint-*")
# checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
# load_checkpoint = checkpoints[-1]
# print(f"last checkpoint: {load_checkpoint}")

#load_checkpoint = "../models/1GPU_run_jup2021-05-22-15_20_57/checkpoint-309504/"
#load_checkpoint = "/storage/brno6/home/ahajek/Summarization/models_ext/noconotr3in3_epoch19_init" # initial checkpoint

#### Setting all training args

In [6]:
os.environ["PBS_NGPUS"]

'0'

In [7]:
now = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
now = now.replace(":","_").replace(" ", "-")
parser = argparse.ArgumentParser()
parser.add_argument("--lr",default=5e-5, type=float, help="learning rate")
parser.add_argument("--seed",default=42, type=int,  help="seed to replicate results")
parser.add_argument("--gradient-accumulation-steps",default=gas, type=int, help="gradient_accumulation_steps")
parser.add_argument("--batch-size",default=bs, type=int,  help="batch_size")
parser.add_argument("--warmup",default=500, type=int,  help="warmup steps for learning rate")
parser.add_argument("--weight-decay",default=0.01, type=float,  help="weight decay rate parameter")
parser.add_argument("--n-gpu",default=os.environ["PBS_NGPUS"], type=int, required=False, help="no of gpu available")
parser.add_argument("--fairscale",default="", type=str, required=False, choices=["simple", "zero_dp_2", "zero_dp_3"], help="GPU paralellization via Fairscale, " +
                    "more info in HuggingFace's Trainer docs")
parser.add_argument("--deepspeed", default=None, type=str, required=False, help="GPU paralellization via Deepspeed, the value is the location of DeepSpeed json config file; " +
                    "more info in HuggingFace's Trainer docs")
parser.add_argument("--num-workers",default=os.environ["PBS_NCPUS"], type=int,  help="num of cpus available")
parser.add_argument("--device",default=torch.device('cuda'), help="torch.device object")
parser.add_argument("--num-train-epochs",default=num_epochs, type=int,  help="number of training epochs")
parser.add_argument("--output-dir",default='/storage/projects/msml/mg_neims_branch/MassGenie/output', type=str,  help="Path to save evaluation results")
parser.add_argument("--save-dir",default='/storage/projects/msml/mg_neims_branch/MassGenie/models', type=str,  help="Path to save trained model")
parser.add_argument("--save-name", type=str, default=f'bart_{now}', help="Name of the model, used for saves")
parser.add_argument("--load-checkpoint", type=str, default='', help="Path to the checkpoint to resume training")
parser.add_argument("--config-dir",default='/storage/projects/msml/mg_neims_branch/MassGenie/configs', type=str,  help="Path to save config files of models")
parser.add_argument("--fp16",default=True, type=bool, required=False, help="whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
parser.add_argument("--max-grad-norm",default=1.0, type=float, help="max gradient norm.")
parser.add_argument("--train-data-path",default=f'/storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/{data_type}_bart_prepared_data_train.pkl', type=str, help="Path to jsonl train dataset")
parser.add_argument("--valid-data-path",default=f'/storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/{data_type}_bart_prepared_data_valid.pkl', type=str, help="Path to jsonl validation dataset")
parser.add_argument("--log-steps",default=50, type=int,  help="number of steps between logs")
parser.add_argument("--eval-steps",default=7142, type=int,  help="number of steps between evaluations")
parser.add_argument("--tokenizer-path",default='/storage/brno6/home/ahajek/nic', type=str, help="location of the desied tokenizer (special sep token will be added))")
parser.add_argument("--model-path",default='/storage/projects/msml/mg_neims_branch/MassGenie/models/NECO', type=str, help="location of the desired model to finetune")
parser.add_argument("--wandb", action='store_true', default=True, help="optinal logging via Weights&Biases")
parser.add_argument("--wandb-resume", action='store_true', default=resume_training, help="resume logging via wandb, needs an valid run ID set in args.wandb-id")
parser.add_argument("--wandb-id", type=str, default=wandb.util.generate_id(), help="Process unique wandb ID used for resumin the training process")
parser.add_argument("--tensorboard", action='store_true', default=False, help="optinal logging via TensorBoard")

args = parser.parse_args([])
arg_log = print_args(args)

# extended outputs
logging.set_verbosity_info()

arguments:
  lr ........................... 5e-05
  seed ......................... 42
  gradient_accumulation_steps .. 16
  batch_size ................... 4
  warmup ....................... 500
  weight_decay ................. 0.01
  n_gpu ........................ 0
  fairscale .................... 
  deepspeed .................... None
  num_workers .................. 3
  device ....................... cuda
  num_train_epochs ............. 8
  output_dir ................... /storage/projects/msml/mg_neims_branch/MassGenie/output
  save_dir ..................... /storage/projects/msml/mg_neims_branch/MassGenie/models
  save_name .................... bart_2022-03-30-18_33_49
  load_checkpoint .............. 
  config_dir ................... /storage/projects/msml/mg_neims_branch/MassGenie/configs
  fp16 ......................... True
  max_grad_norm ................ 1.0
  train_data_path .............. /storage/projects/msml/mg_neims_branch/MassGenie/data/trial_set/1M_bart_prepared_data

#### Loading data, tokenizer, model

In [8]:
# BART CONIGURATION
# chytre vocab size, ale nemame tokenizer: # 500+len(tokenizer.get_ids()),
# dal jsem vocab_size na 600, pro jistotu (581 by melo stacit)
config = BartConfig(vocab_size = 600,
                             max_position_embeddings = SEQ_LEN,
                             encoder_layers = 12,
                             encoder_ffn_dim = 4096,
                             encoder_attention_heads = 16,
                             decoder_layers = 12,
                             decoder_ffn_dim = 4096,
                             decoder_attention_heads = 16,
                             encoder_layerdrop = 0.0,
                             decoder_layerdrop = 0.0,
                             activation_function = 'gelu',
                             d_model = 1024,
                             dropout = 0.2,
                             attention_dropout = 0.0,
                             activation_dropout = 0.0,
                             init_std = 0.02,
                             classifier_dropout = 0.0,
                             scale_embedding = False,
                             use_cache = True,
                             num_labels = 3,
                             pad_token_id = 501,
                             bos_token_id = 503,
                             eos_token_id = 502,
                             is_encoder_decoder = True,
                             decoder_start_token_id = 502,
                             forced_eos_token_id = 502,
                             static_position_embeddings=True
                   )


In [10]:
# DATA
train_data = SpectroDataset(args.train_data_path)
valid_data = SpectroDataset(args.valid_data_path)

# clean memory
gc.collect()
torch.cuda.empty_cache()

# TOKENIZER
# tokenizer = add_special_tokens(args.tokenizer_path)

# MODEL
model = BartForConditionalGeneration(config)
# model = BartForConditionalGeneration.from_pretrained(args.model_path)

# model.to(args.device)
model.to("cpu")
print("kratkej vypis :D")

kratkej vypis :D


In [10]:
# for 1M dataset
train_data.data.rename(columns={"lm_labels":"labels"}, inplace=True)
valid_data.data.rename(columns={"lm_labels":"labels"}, inplace=True)
train_data.data["position_ids"] = range(len(train_data))

In [11]:
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.input_ids])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.decoder_input_ids])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.encoder_attention_mask])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.decoder_attention_mask])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.labels])
assert 0 == sum([len(x) != SEQ_LEN for x in train_data.data.position_ids])
train_data.data

,destereo_smiles,input_ids,decoder_input_ids,encoder_attention_mask,decoder_attention_mask,labels,position_ids
365667,CCN(C(=O)CNC1CN(CC1c1cncn1C)C(=O)C(=O)N)C1CC1,"[30, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4...","[500, 531, 531, 541, 506, 531, 506, 510, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 531, 531, 541, 506, 531, 506, 510, 542,...","[4, 1, 6, 5, 8, 9, 8, 9, 7, 6, 5, 3, 2, 2, 5, ..."
392866,O=C(CN1CCNC(=O)C1)NCCC1CN(CC1C)Cc1nncn1C,"[33, 36, 39, 40, 41, 42, 43, 44, 46, 51, 52, 5...","[500, 542, 510, 531, 506, 531, 541, 520, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 510, 531, 506, 531, 541, 520, 531,...","[1, 4, 1, 2, 8, 9, 6, 8, -9223372036854775799,..."
243721,COCn1nccc1S(=O)(=O)N1CCCC1C(=O)NCC(=O)OC,"[33, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 5...","[500, 531, 542, 531, 565, 520, 565, 555, 555, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 531, 542, 531, 565, 520, 565, 555, 555,...","[6, 6, 3, 9, 8, 8, 8, 9, 7, 5, 6, 6, 8, 8, 9, ..."
102237,CCn1nncc1C(=O)N1CCOC2C1CN(C2)C(=O)c1nnn(n1)C,"[28, 29, 33, 34, 36, 37, 38, 39, 40, 41, 42, 4...","[500, 531, 531, 565, 520, 565, 565, 555, 555, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 531, 531, 565, 520, 565, 565, 555, 555,...","[1, -9223372036854775799, 5, 2, 6, 5, 5, 7, 7,..."
247067,CCn1ccc(n1)C(=O)NC1CN(CC1c1ncn(c1)C)C(=O)C(=O)N,"[33, 36, 38, 39, 40, 41, 42, 43, 44, 45, 49, 5...","[500, 531, 531, 565, 520, 555, 555, 555, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 531, 531, 565, 520, 555, 555, 555, 506,...","[1, 1, 3, 7, 6, 8, 8, 6, 8, 4, 2, 4, 5, 6, 7, ..."
...,...,...,...,...,...,...,...
92540,OC1CN(CC1NC(=O)C(c1cnc[nH]1)(C)C)C(=O)c1c[nH]c...,"[39, 40, 41, 42, 43, 44, 45, 46, 51, 52, 53, 5...","[500, 542, 531, 520, 531, 541, 506, 531, 531, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 520, 531, 541, 506, 531, 531,...","[5, 4, 9, 8, 8, 8, 5, -9223372036854775799, 4,..."
238810,OC(CN(C(=O)Cn1ccc(cc1=O)C)C)CNC(=O)C1CC1C(=O)N,"[30, 33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 4...","[500, 542, 531, 506, 531, 541, 506, 531, 506, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 531, 506, 531, 541, 506, 531, 506,...","[5, 2, 1, 2, 8, 6, 9, 8, 8, 9, 7, 5, 5, 7, 8, ..."
311793,O=C(Nc1ccc(c(c1)S(=O)(=O)N)C)CNS(=O)(=O)N(C)C,"[16, 27, 28, 30, 31, 33, 34, 36, 39, 40, 41, 4...","[500, 542, 510, 531, 506, 541, 555, 520, 555, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 542, 510, 531, 506, 541, 555, 520, 555,...","[1, 3, 2, 6, 3, 2, 1, 3, 7, 5, 6, 8, 6, 9, 7, ..."
131110,NCC(C(=O)NC1CC2CCC(C1)N2C(=O)c1cc(=O)n2c(n1)nn...,"[30, 33, 39, 40, 41, 42, 43, 44, 45, 46, 49, 5...","[500, 541, 531, 531, 506, 531, 506, 510, 542, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 541, 531, 531, 506, 531, 506, 510, 542,...","[4, -9223372036854775799, 7, 5, 9, 8, 8, 8, 5,..."


In [12]:
print(f"Number of model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Number of model parameters: 353746944


#### Setting run resuming and WandB logging

In [13]:
# Resume training
if resume_training:
    args.load_checkpoint = load_checkpoint
    if args.wandb_resume:
        args.wandb_id = resume_wandb_id

# Init wandb
if args.wandb:
    ! wandb login
    %env WANDB_PROJECT = BART_for_gcms
    wandb.init(id=args.wandb_id, resume="allow")
    wandb.run.name = args.save_name

wandb: Currently logged in as: hajekad (use `wandb login --relogin` to force relogin)
env: WANDB_PROJECT=BART_for_gcms


wandb: Currently logged in as: hajekad (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


#### Setting training arguments (according to args)

In [14]:
training_args = TrainingArguments(
    output_dir=args.save_dir+"/"+args.save_name,                            # output directory
    num_train_epochs=args.num_train_epochs,              # total # of training epochs
    per_device_train_batch_size=args.batch_size,         # batch size per device during training
    per_device_eval_batch_size=args.batch_size,          # batch size for evaluation
    gradient_accumulation_steps=args.gradient_accumulation_steps,
    warmup_steps=args.warmup,                                    # number of warmup steps for learning rate scheduler
    weight_decay=args.weight_decay,                                   # strength of weight decay
    logging_dir=args.save_dir + './logs',                # directory for storing logs
    report_to=["wandb"],
    run_name=args.save_name,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
#     eval_steps=args.eval_steps,
    logging_steps=2, # args.log_steps,
    save_strategy="epoch",
#     save_steps=1000,
    fp16=args.fp16,
    dataloader_drop_last=True,
    save_total_limit=2,
    dataloader_num_workers=args.num_workers,
    sharded_ddp=args.fairscale,
    deepspeed=args.deepspeed 
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=valid_data,             # evaluation dataset
    data_collator = SpectroDataCollator()
)

PyTorch: setting up devices
Using amp half precision backend


### Training

In [ ]:
# log args to "args" file in args.save_dir (right before the training starts, to be the latest)
Path(f'{args.save_dir}/{args.save_name}').mkdir(exist_ok=True)
with open(f'{args.save_dir}/{args.save_name}/args', 'w+') as output_file:
    output_file.write(arg_log)

if args.load_checkpoint:
    trainer.train(args.load_checkpoint)
else:
    #trainer.evaluate()
    trainer.train()

/storage/brno6/home/ahajek/.local-Pytorch-21.SIF/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 354766
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 44344
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:752: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use re

Epoch,Training Loss,Validation Loss
0,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 44346
  Batch size = 4
Saving model checkpoint to /storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-03-30-10_22_16/checkpoint-5543
Configuration saved in /storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-03-30-10_22_16/checkpoint-5543/config.json
Model weights saved in /storage/projects/msml/mg_neims_branch/MassGenie/models/bart_2022-03-30-10_22_16/checkpoint-5543/pytorch_model.bin
